In [1]:
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
load_dotenv("../.env", override=True)

True

In [2]:
@tool
def multiply_triple(number):
    """
    Accepts a number, multiplies it by 3 and returns back
    """
    return number * 3

In [3]:
tool_list = [TavilySearchResults(), multiply_triple]

In [4]:
llm = ChatOpenAI(model = "gpt-4o", temperature = 0.).bind_tools(tool_list)

In [5]:
SYSTEM_MESSAGE = """
You are a helpful assistant that can use tools to answer questions
"""

In [6]:
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

In [7]:
def run_agent_reasoning(state: MessagesState) -> MessagesState:
    """
    Run the agent reasoning node
    """
    response = llm.invoke([{"role": "system", "content": SYSTEM_MESSAGE}, *state['messages']])
    return {"messages": [response]}

In [8]:
tool_node = ToolNode(tool_list)

In [9]:
AGENT_REASONING = "agent_reason"
ACT = "act"
LAST = -1

In [10]:
flow = StateGraph(MessagesState)

In [11]:
flow.add_node("agent_reason", run_agent_reasoning)

In [12]:
flow.add_node("act", tool_node)

In [13]:
flow.add_edge(START, "agent_reason")

In [14]:
def should_continue(state: MessagesState):
    if not state["messages"][-1].tool_calls:
        return END

    return "act"

In [15]:
flow.add_edge("act", "agent_reason")

In [16]:
flow.add_conditional_edges("agent_reason", should_continue, {
    END:END, 
    "act":"act"}
)

In [17]:
app = flow.compile()

In [18]:
app.get_graph().draw_mermaid_png(output_file_path = "flow_State.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xcb\x00\x00\x00\xf9\x08\x02\x00\x00\x00\x06+V\xc2\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x9c\xed\x9dy\\\x13\xd7\xfa\xffO\xf6\x90\x15B \xec\xbb\x02\x82\x82\x06D\x8b\xb7\xae\xa8uA\xc1\xa5\x08Z\x97.z[\xad\xd7\xba}om]j{[\xd7\xdeVo\xad^\xac;\xd5j\xb5\xae\xd5*(jQ\xa9uE\xc0\x85}\xdf\xb3\'$\x93\xcc\xef\x8f\xf8C\xea\r\x888\x93\x99$\xe7\xfd\xf2\x8f$3\xf3\x9c\x8f\xc9\x87s\x9e9s\x16\n\x8a\xa2\x00\x02\xc1\r*\xd1\x02 v\x0et\x18\x04_\xa0\xc3 \xf8\x02\x1d\x06\xc1\x17\xe80\x08\xbe@\x87A\xf0\x85N\xb4\x00\xb2\xa0lA\xe4\x8d\x06\xb5\x02Q+\x10\xc4\x80\xa2F\xa2\x05u\x01&\x9b\xca\xe6R\xb9\x02:\xdf\x85\xee"a\x12-\xc72\x14\x07\xef\x0fk\xac\xd2\x17\xddW\x95\xe4\xa9\xd8\\:@Q\x8e\x80\xce\x15\xd2\x9c\xb8t#b"ZZ\x97P4#j9\xc2\xe2\xd0\xea\xca\xb4\x01\x11\xdc\xa0H\x9eO\x0f\'\xa2E\xfd\x05\xc7u\x98\xb2\x05\xc99\xd9H\xa1Q\x9c\xdd\x18A\x11\\\xb17\x8bhE\xaf\x84\xa2\x19)\xc9S5\xd5\x1aZj[_\x1b/\xf6\x0cd\x13\xad\xe8)\x0e\xea\xb0?\xce5\xe7\xe7*^\x1b\'\xee\xd1\x

In [19]:
res = app.invoke({"messages": [HumanMessage(content = "What is the temperature in Tokyo, Triple it and return back the result")]})

In [20]:
res["messages"]#.tool_calls

[HumanMessage(content='What is the temperature in Tokyo, Triple it and return back the result', additional_kwargs={}, response_metadata={}, id='fda15cd8-5ca9-4c9b-a47a-4716ab51a1c3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nFYnmP0AO87BoJCfFVEr06Ci', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 137, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BiGzhWPjm5ZNhHDUW5FYMzD2HZU5x', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--34630c03-0d84-40ce-9a25-00af6eefa913-0', tool_calls=[